 # Курсовая работа
 ## на тему "Датасеты"

1) Выбор своего датасета (условие — выполнено в виде интерактивной презентации в юпитере)

Нужно попытаться визуализировать экземпляр данных

2) найти публикацию\статью про датасет, внедрить ссылку и кратко описать в презентации (+ какие популярные модели используются)

3) Пример данных с разметкой + пример кода для загрузки тестового набора

4) Пример применения готовой модели на этих данных

!) Обязательное использование Markdown + LaTeX

!) Обязательное использование Plotly

!) Обязательное использование готового кода с алгоритмом относительно датасета

!) Обязательно воспользоваться доплнительно TensorBoard или аналогом

*) (realesrgan, pifuhd) Препроцессинг другой моделью (+ описать)

Титульник надо напечатать + ссылка на ноутбук

## 1 + 2) Выбор Датасета, Публикация\Статья про датасет, ссылка, краткое описание
Мой выбор пал на датасет Reddit. Ссылка: https://paperswithcode.com/dataset/reddit

Данный датасет представляет собой датасет графов, состоящий из постов сайта Reddit, сделанного в месяце сентября 2014 года. Данные разделены на сообщества или "Суб-Реддиты", каждому из которых принадлежит своё сообщение. Сообщения были взяты из 50 крупнейших сообществ и были построены в графике "Публикация-к-публикации", соединяющие посты одного пользователя. В общей сложности датасет состоит из 232 965 постов в средней степенью 492. Первые 20 дней используются для обучения, остальные дни - для тестирования (из них 30% используются для валидации). Для моделей подготовлены 300-мерные векторы слов GloVe CommonCrawl.

## 3) Пример данных с разметкой + пример кода для загрузки тестового набора

In [20]:
import os
import os.path as osp
from typing import Callable, List, Optional

import numpy as np
import scipy.sparse as sp
import torch

from torch_geometric.data import (
    Data,
    InMemoryDataset,
    download_url,
    extract_zip,
)
from torch_geometric.utils import coalesce


class Reddit(InMemoryDataset):
    r"""The Reddit dataset from the `"Inductive Representation Learning on
    Large Graphs" <https://arxiv.org/abs/1706.02216>`_ paper, containing
    Reddit posts belonging to different communities.

    Args:
        root (string): Root directory where the dataset should be saved.
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)

    Stats:
        .. list-table::
            :widths: 10 10 10 10
            :header-rows: 1

            * - #nodes
              - #edges
              - #features
              - #classes
            * - 232,965
              - 114,615,892
              - 602
              - 41
    """

    url = 'https://data.dgl.ai/dataset/reddit.zip'

    def __init__(
        self,
        root: str,
        transform: Optional[Callable] = None,
        pre_transform: Optional[Callable] = None,
    ):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self) -> List[str]:
        return ['reddit_data.npz', 'reddit_graph.npz']

    @property
    def processed_file_names(self) -> str:
        return 'data.pt'

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        os.unlink(path)

    def process(self):
        data = np.load(osp.join(self.raw_dir, 'reddit_data.npz'))
        x = torch.from_numpy(data['feature']).to(torch.float)
        y = torch.from_numpy(data['label']).to(torch.long)
        split = torch.from_numpy(data['node_types'])

        adj = sp.load_npz(osp.join(self.raw_dir, 'reddit_graph.npz'))
        row = torch.from_numpy(adj.row).to(torch.long)
        col = torch.from_numpy(adj.col).to(torch.long)
        edge_index = torch.stack([row, col], dim=0)
        edge_index = coalesce(edge_index, num_nodes=x.size(0))

        data = Data(x=x, edge_index=edge_index, y=y)
        data.train_mask = split == 1
        data.val_mask = split == 2
        data.test_mask = split == 3

        data = data if self.pre_transform is None else self.pre_transform(data)

        torch.save(self.collate([data]), self.processed_paths[0])

ImportError: cannot import name 'Tensor' from 'torch' (unknown location)

## 4) Пример применения готовой модели на этих данных